# Processing Data

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from math import radians, sin, cos, asin, sqrt
import numpy as np

sc = SparkContext.getOrCreate()
ss = SparkSession.builder.getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
9,application_1579045319375_0010,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# s3 access key

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
endo_df = ss.read.json('s3://usfca-endomondo/endomondoHR_proper.json')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
endo_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- altitude: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- gender: string (nullable = true)
 |-- heart_rate: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- id: long (nullable = true)
 |-- latitude: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- longitude: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- speed: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- sport: string (nullable = true)
 |-- timestamp: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- url: string (nullable = true)
 |-- userId: long (nullable = true)

In [5]:
endo_df = endo_df.where("sport in ('bike', 'run', 'bike (transport)', 'mountain bike')")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
endo_rdd = endo_df.rdd

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
def my_haversine(pos1, pos2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    lon1, lat1 = pos1
    lon2, lat2 = pos2
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 3956 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
miles = (endo_rdd.map(lambda x: (x['id'], list(zip(x['latitude'], x['longitude']))))
 .mapValues(lambda x: [my_haversine(x[i], x[i+1]) for i in range(len(x)-1)])
 .mapValues(lambda x: sum(x)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
miles_df = miles.toDF()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
miles_df = miles_df.withColumnRenamed(miles_df.columns[0], 'id').withColumnRenamed(miles_df.columns[1], 'miles')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
miles_df.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-----------------+
|       id|            miles|
+---------+-----------------+
|396826535|52.29380964306281|
+---------+-----------------+
only showing top 1 row

In [12]:
joined = endo_df.join(miles_df, 'id', 'inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
joined.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+------+--------------------+--------------------+--------------------+-----+-----+--------------------+--------------------+------+-----------------+
|     id|            altitude|gender|          heart_rate|            latitude|           longitude|speed|sport|           timestamp|                 url|userId|            miles|
+-------+--------------------+------+--------------------+--------------------+--------------------+-----+-----+--------------------+--------------------+------+-----------------+
|9408771|[32.389, 32.556, ...|  male|[104, 115, 125, 1...|[56.235583, 56.23...|[9.08747, 9.08841...| null| bike|[1303313589, 1303...|https://www.endom...| 65547|58.48725698600932|
+-------+--------------------+------+--------------------+--------------------+--------------------+-----+-----+--------------------+--------------------+------+-----------------+
only showing top 1 row

In [14]:
joined.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: long (nullable = true)
 |-- altitude: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- gender: string (nullable = true)
 |-- heart_rate: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- latitude: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- longitude: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- speed: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- sport: string (nullable = true)
 |-- timestamp: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- url: string (nullable = true)
 |-- userId: long (nullable = true)
 |-- miles: double (nullable = true)

In [15]:
# sc.install_pypi_package('numpy')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
avg_heart_df = endo_rdd.map(lambda x: (x['id'], x['heart_rate'])).mapValues(lambda x: np.mean(x)).mapValues(lambda x: float(x))
var_heart_df = endo_rdd.map(lambda x: (x['id'], x['heart_rate'])).mapValues(lambda x: np.var(x)).mapValues(lambda x: float(x))
std_heart_df = endo_rdd.map(lambda x: (x['id'], x['heart_rate'])).mapValues(lambda x: np.std(x)).mapValues(lambda x: float(x))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
avg_heart_df = avg_heart_df.toDF()
var_heart_df = var_heart_df.toDF()
std_heart_df = std_heart_df.toDF()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
avg_heart_df = avg_heart_df.withColumnRenamed(avg_heart_df.columns[0], 'id').withColumnRenamed(avg_heart_df.columns[1], 'avg_heart')
var_heart_df = var_heart_df.withColumnRenamed(var_heart_df.columns[0], 'id').withColumnRenamed(var_heart_df.columns[1], 'var_heart')
std_heart_df = std_heart_df.withColumnRenamed(std_heart_df.columns[0], 'id').withColumnRenamed(std_heart_df.columns[1], 'std_heart')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
joined = joined.join(avg_heart_df, 'id', 'inner')
joined = joined.join(var_heart_df, 'id', 'inner')
joined = joined.join(std_heart_df, 'id', 'inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
joined.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: long (nullable = true)
 |-- altitude: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- gender: string (nullable = true)
 |-- heart_rate: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- latitude: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- longitude: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- speed: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- sport: string (nullable = true)
 |-- timestamp: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- url: string (nullable = true)
 |-- userId: long (nullable = true)
 |-- miles: double (nullable = true)
 |-- avg_heart: double (nullable = true)
 |-- var_heart: double (nullable = true)
 |-- std_heart: double (nullable = true)

In [21]:
id_min_alt_df = (endo_rdd.map(lambda x: (x['id'], x['altitude']))
               .mapValues(lambda x: np.min(x))
               .mapValues(lambda x: float(x)).toDF())

id_max_alt_df = (endo_rdd.map(lambda x: (x['id'], x['altitude']))
               .mapValues(lambda x: np.max(x))
               .mapValues(lambda x: float(x)).toDF())

id_mean_alt_df = (endo_rdd.map(lambda x: (x['id'], x['altitude']))
                .mapValues(lambda x: np.mean(x))
                .mapValues(lambda x: float(x)).toDF())

id_var_alt_df = (endo_rdd.map(lambda x: (x['id'], x['altitude']))
                .mapValues(lambda x: np.var(x))
                .mapValues(lambda x: float(x)).toDF())

id_std_alt_df = (endo_rdd.map(lambda x: (x['id'], x['altitude']))
                .mapValues(lambda x: np.std(x))
                .mapValues(lambda x: float(x)).toDF())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
id_min_alt_df = id_min_alt_df.withColumnRenamed(id_min_alt_df.columns[0], 'id').withColumnRenamed(id_min_alt_df.columns[1], 'min_alt')
id_max_alt_df = id_max_alt_df.withColumnRenamed(id_max_alt_df.columns[0], 'id').withColumnRenamed(id_max_alt_df.columns[1], 'max_alt')
id_mean_alt_df = id_mean_alt_df.withColumnRenamed(id_mean_alt_df.columns[0], 'id').withColumnRenamed(id_mean_alt_df.columns[1], 'avg_alt')
id_var_alt_df = id_var_alt_df.withColumnRenamed(id_var_alt_df.columns[0], 'id').withColumnRenamed(id_var_alt_df.columns[1], 'var_alt')
id_std_alt_df = id_std_alt_df.withColumnRenamed(id_std_alt_df.columns[0], 'id').withColumnRenamed(id_std_alt_df.columns[1], 'std_alt')



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
joined = joined.join(id_min_alt_df, 'id', 'inner')
joined = joined.join(id_max_alt_df, 'id', 'inner')
joined = joined.join(id_mean_alt_df, 'id', 'inner')
joined = joined.join(id_var_alt_df, 'id', 'inner')
joined = joined.join(id_std_alt_df, 'id', 'inner')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
joined.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: long (nullable = true)
 |-- altitude: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- gender: string (nullable = true)
 |-- heart_rate: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- latitude: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- longitude: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- speed: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- sport: string (nullable = true)
 |-- timestamp: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- url: string (nullable = true)
 |-- userId: long (nullable = true)
 |-- miles: double (nullable = true)
 |-- avg_heart: double (nullable = true)
 |-- var_heart: double (nullable = true)
 |-- std_heart: double (nullable = true)
 |-- min_alt: double (nullable = true)
 |-- max_alt: double (nullable = true)
 |-- avg_alt: double (nullable = true)
 |-- var_alt: double 

In [25]:
joined = joined.drop('latitude', 'longitude', 'speed', 'url', 'userId', 'altitude', 'heart_rate')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Write to HDFS

In [28]:
joined.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+-----+--------------------+-----------------+---------+----------+------------------+-------+-------+---------+----------------+------------------+
|     id|gender|sport|           timestamp|            miles|avg_heart| var_heart|         std_heart|min_alt|max_alt|  avg_alt|         var_alt|           std_alt|
+-------+------+-----+--------------------+-----------------+---------+----------+------------------+-------+-------+---------+----------------+------------------+
|9408771|  male| bike|[1303313589, 1303...|58.48725698600932|  130.674|290.511724|17.044404477716434| 27.841| 86.003|57.097864|226.418976249504|15.047224868709312|
+-------+------+-----+--------------------+-----------------+---------+----------+------------------+-------+-------+---------+----------------+------------------+
only showing top 1 row

In [29]:
joined.write.save("s3://usfca-endomondo/processed_data", format='parquet', mode='append')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Read from HDFS

In [30]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
data = sqlContext.read.format('parquet').load("s3://usfca-endomondo/processed_data") 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
data.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+-------------+--------------------+------------------+---------+------------------+------------------+-------+-------+------------------+------------------+------------------+
|      id|gender|        sport|           timestamp|             miles|avg_heart|         var_heart|         std_heart|min_alt|max_alt|           avg_alt|           var_alt|           std_alt|
+--------+------+-------------+--------------------+------------------+---------+------------------+------------------+-------+-------+------------------+------------------+------------------+
| 7864389|  male|         bike|[1300766553, 1300...|20.284106160771454|  135.554|        171.915084|13.111639256782503|704.354|760.232| 733.4767239999999|144.97629998782398|12.040610449135208|
|17091626|  male|         bike|[1311619064, 1311...| 42.29974076736979|  143.166|155.75844400000003|12.480322271480013|   86.2|  131.0|106.88680000000001| 87.74198575999999|  9.36706921934497|
|18006064|  male|         bike|[131